## Gap Annotation to Bed

In [104]:
import HTSeq
import pandas as pd


def single_sequence(sequence):
    '''
    sequence       refers to a HTSeq.Sequence object
    
        the function returns DataFrame object in GFF3 format 
    '''
    seq = sequence.seq.decode()
    
    bases = ['A','a','C','c','G','g','T','t']

    start = []
    end = []

    above = None
    for i in range(len(seq)):
        if seq[i] in bases and above == 'N':
            end.append(i+1)
        elif seq[i] == 'N' and above != 'N':
            start.append(i+1)
        above = seq[i]

    if seq[-1] == 'N':
        end.append(len(seq))
        
    gff3 = pd.DataFrame({'name':sequence.name, 'source':'.','type':'BAC', \
                         'start':start, 'end':end, 'score':'.', \
                         'strand':'+', 'phase':'.', 'attribute':['ID=gap%05d' % i for i in range(1,len(start)+1)]})
    
    return gff3

def annotateGap(genome_file, gff3_file):
    GFF3 = pd.DataFrame()
    for i in HTSeq.FastaReader(genome_file):
        gff3 = single_sequence(i)
        GFF3 = pd.concat([GFF3,gff3],ignore_index=True)
        
    GFF3.to_csv(gff3_file, sep='\t', header=None, index=None)
    return GFF3

if __name__ == '__main__':
    annotateGap('seq.fa','seq.gff3')


,name,source,type,start,end,score,strand,phase,attribute
0,a,.,BAC,11,12,.,+,.,ID=gap00001
1,a,.,BAC,19,22,.,+,.,ID=gap00002
2,a,.,BAC,32,35,.,+,.,ID=gap00003
3,b,.,BAC,8,10,.,+,.,ID=gap00001
4,b,.,BAC,11,12,.,+,.,ID=gap00002
5,b,.,BAC,21,25,.,+,.,ID=gap00003
6,b,.,BAC,34,38,.,+,.,ID=gap00004
